```{contents}
```
## **Tool-Using Agent in LangGraph**

A **Tool-Using Agent** in LangGraph is an autonomous execution unit that **reasons about a task, selects external tools, invokes them, observes their outputs, and iterates until a goal is achieved** — all within a **stateful, cyclic graph**.

It operationalizes the classical **ReAct (Reason → Act → Observe)** paradigm.

---

### **1. Why Tool-Using Agents Exist**

LLMs alone cannot:

* access real-time data
* perform reliable computation
* modify external systems
* retrieve private knowledge

Tool-using agents extend LLMs with **action capability**.

---

### **2. Conceptual Workflow**

```
User Goal
   ↓
Reason (LLM)
   ↓
Select Tool
   ↓
Execute Tool
   ↓
Observe Result
   ↓
Update State
   ↓
Repeat or Finish
```

This forms a **cyclic LangGraph**.

---

### **3. Core Components in LangGraph**

| Component   | Role                        |
| ----------- | --------------------------- |
| State       | Shared memory between steps |
| Reason Node | LLM decides next action     |
| Tool Node   | Executes selected tool      |
| Router      | Controls loop continuation  |
| Termination | Stop condition              |

---

### **4. State Design**

```python
class AgentState(TypedDict):
    messages: list
    tool_result: str
    done: bool
```

---

### **5. Building a Tool-Using Agent**

### Step 1 — Define Tools

```python
from langchain.tools import tool

@tool
def search(query: str) -> str:
    return f"Search results for {query}"
```

---

### Step 2 — Reasoning Node (LLM)

```python
def reason(state):
    response = llm.invoke(state["messages"])
    return {"messages": state["messages"] + [response]}
```

---

### Step 3 — Tool Execution Node

```python
def act(state):
    tool_call = state["messages"][-1].tool_calls[0]
    result = tools[tool_call["name"]].invoke(tool_call["args"])
    return {"tool_result": result}
```

---

### Step 4 — Observation Node

```python
def observe(state):
    state["messages"].append({"role": "tool", "content": state["tool_result"]})
    return {}
```

---

### Step 5 — Loop Control

```python
def route(state):
    if "FINAL ANSWER" in state["messages"][-1].content:
        return END
    return "reason"
```

---

### Step 6 — Assemble Graph

```python
from langgraph.graph import StateGraph, END

builder = StateGraph(AgentState)

builder.add_node("reason", reason)
builder.add_node("act", act)
builder.add_node("observe", observe)

builder.set_entry_point("reason")
builder.add_edge("reason", "act")
builder.add_edge("act", "observe")

builder.add_conditional_edges("observe", route, {
    "reason": "reason",
    END: END
})

agent = builder.compile()
```

---

### **6. Execution Example**

```python
result = agent.invoke({"messages": [{"role": "user", "content": "Find GDP of India"}], "done": False})
```

The agent will:

```
Reason → Choose search → Execute → Observe → Reason → Answer → Stop
```

---

### **7. Variants of Tool-Using Agents**

| Variant                | Purpose                    |
| ---------------------- | -------------------------- |
| Single-Tool Agent      | One specialized capability |
| Multi-Tool Agent       | Choose among many tools    |
| Planner-Executor Agent | Plan then act              |
| Self-Reflecting Agent  | Evaluate tool output       |
| Human-in-Loop Agent    | Requires approval          |
| Autonomous Agent       | Long-running execution     |

---

### **8. Safety & Production Controls**

| Control         | Purpose               |
| --------------- | --------------------- |
| Tool sandboxing | Prevent misuse        |
| Cost limits     | Prevent runaway loops |
| Recursion limit | Avoid infinite loops  |
| Human approval  | High-risk tools       |
| Audit logging   | Compliance            |

```python
agent.invoke(input, config={"recursion_limit": 10})
```

---

### **9. Why LangGraph is Ideal for Tool-Using Agents**

LangGraph provides:

* explicit state control
* deterministic execution
* safe cyclic workflows
* persistent memory
* human oversight

This makes tool-using agents **robust, auditable, and production-ready**.


### Demonstration

In [3]:
# ---- ONE CELL: Tool-Using Agent with LangGraph ----

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage

# -------- 1. Define State --------
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

# -------- 2. Define Tool --------
@tool
def search(query: str) -> str:
    """Search tool that returns information about the query."""
    # Mock implementation - in real scenario, this would call an API
    if "GDP" in query or "India" in query:
        return "India's GDP is approximately $3.7 trillion USD (2023)"
    return f"Search results for: {query}"

# -------- 3. LLM --------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools([search])

# -------- 4. Nodes --------
def reason_and_act(state: AgentState):
    """LLM reasons and decides whether to use tools or provide final answer."""
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

def execute_tools(state: AgentState):
    """Execute any tool calls from the last message."""
    last_message = state["messages"][-1]
    tool_calls = last_message.tool_calls
    
    tool_messages = []
    for tool_call in tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        
        # Execute the tool
        if tool_name == "search":
            result = search.invoke(tool_args)
        else:
            result = f"Unknown tool: {tool_name}"
        
        # Create tool message
        tool_messages.append(
            ToolMessage(content=result, tool_call_id=tool_call["id"])
        )
    
    return {"messages": tool_messages}

def should_continue(state: AgentState):
    """Determine if we should continue the loop or end."""
    last_message = state["messages"][-1]
    
    # If the last message has tool calls, continue to execute them
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    
    # Otherwise, we're done
    return END

# -------- 5. Build Graph --------
builder = StateGraph(AgentState)

builder.add_node("agent", reason_and_act)
builder.add_node("tools", execute_tools)

builder.set_entry_point("agent")

builder.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tools": "tools",
        END: END
    }
)

builder.add_edge("tools", "agent")

agent = builder.compile()

# -------- 6. Run --------
print("=== Tool-Using Agent Execution ===\n")

result = agent.invoke({
    "messages": [HumanMessage(content="What is the GDP of India?")]
})

print("Conversation trace:")
for i, msg in enumerate(result["messages"], 1):
    role = msg.__class__.__name__
    content = msg.content if hasattr(msg, 'content') else str(msg)
    print(f"\n{i}. {role}:")
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        print(f"   Tool calls: {msg.tool_calls}")
    else:
        print(f"   {content[:200]}")

print("\n" + "="*60)
print("FINAL ANSWER:")
print("="*60)
print(result["messages"][-1].content)

=== Tool-Using Agent Execution ===

Conversation trace:

1. HumanMessage:
   What is the GDP of India?

2. AIMessage:
   Tool calls: [{'name': 'search', 'args': {'query': 'GDP of India 2023'}, 'id': 'call_CW9OuydYJR1Uc0ru0gmSTiZu', 'type': 'tool_call'}]

3. ToolMessage:
   India's GDP is approximately $3.7 trillion USD (2023)

4. AIMessage:
   As of 2023, India's GDP is approximately $3.7 trillion USD.

FINAL ANSWER:
As of 2023, India's GDP is approximately $3.7 trillion USD.
